In [32]:
!pip install -u scipy==1.2.0



Usage:   
  C:\Users\Sherwin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  C:\Users\Sherwin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  C:\Users\Sherwin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install [options] [-e] <vcs project url> ...
  C:\Users\Sherwin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install [options] [-e] <local project path> ...
  C:\Users\Sherwin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install [options] <archive url/path> ...

no such option: -u


In [33]:
!pip install tensorflow==1.15
import tensorflow as tf
print(tf.__version__)



Defaulting to user installation because normal site-packages is not writeable
2.17.0


ERROR: Could not find a version that satisfies the requirement tensorflow==1.15 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.18.0rc0, 2.18.0rc1)
ERROR: No matching distribution found for tensorflow==1.15


In [34]:
import keras
keras.__version__

'3.5.0'

In [35]:
from keras.preprocessing.image import load_img, img_to_array

# This is the path to the image you want to transform.
target_image_path = 'swan.jpg'
# This is the path to the style image.
style_reference_image_path = 'style.jpg'

# Dimensions of the generated picture.
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)


In [36]:
import numpy as np
from keras.applications import vgg19

def preprocess_image (image_path):
    img = load_img(image_path, target_size = (img_height, img_width))
    img  = img_to_array(img)
    img=  np.expand_dims(img, axis=0)
    img= vgg19.preprocess_input(img)
    return img 

def deprocess_image(x):
    x[:,:,0]+= 103.939
    x[:,:,1]+= 116.779
    x[:,:,2]+= 123.68
    
    x=x[:,:,::-1]
    x=np.clip(x, 0, 255).astype('unit8')
    return x

In [48]:
import tensorflow as tf
from tensorflow.keras.applications import vgg19
import tensorflow.keras.backend as K

target_image = preprocess_image(target_image_path)
style_reference_image = preprocess_image(style_reference_image_path)
combination_image = tf.Variable(preprocess_image(target_image_path))

# Combine the three images into a single batch
input_tensor = tf.concat([target_image, style_reference_image, combination_image], axis=0)

input_shape = target_image.shape[1:]  # excluding the batch dimension
input_layer = tf.keras.Input(shape=input_shape)

# Build the VGG19 model using the symbolic input layer and ImageNet weights
model = vgg19.VGG19(input_tensor=input_layer, weights='imagenet', include_top=False)

print('Model loaded.')


Model loaded.


In [49]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [50]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [51]:
def total_variation_loss(x):
    a = K.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [52]:
# Dict mapping layer names to activation tensors
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# Name of layer used for content loss
content_layer = 'block5_conv2'

# Name of layers used for style loss
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

# Weights in the weighted average of the loss components 
total_variation_weight = 1e-4 
style_weight = 1.
content_weight = 0.025

# Define the loss by adding all components to a loss variable 
loss = K.constant(0.)

layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]

loss = loss + content_weight * content_loss(target_image_features, combination_features)

for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl

loss += total_variation_weight * total_variation_loss(combination_image)

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
